# 벡터 DB : Chroma vs Pinecone
- Chroma : 인메모리 vector DB, 로컬메모리 vector DB
- Pinecone : 클라우드 vector DB
    (Pinecone console에 api key 생성 -> .env

# 0. 패키지 설치

In [2]:
%pip install -q pinecone-client langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


# 1. Knowledge Base 구성을 위한 데이터 생성

In [27]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [28]:
len(document_list)

183

In [29]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
# https://python.langchain.com/v0.2/docs/integrations/text_embedding/upstage/#usage

load_dotenv()
embedding = UpstageEmbeddings(
    model="solar-embedding-1-large"
#     model="embedding-query"
)

In [30]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone()
# 데이터를 처음 업로드할 때
index_name = "tax-index-upstage"
database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name=index_name
)
# 업로드한 벡터DB 가져올 때
# database = PineconeVectorStore(
#     embedding=embedding, # 질문을 임베딩하여 유사도 검색
#     index_name=index_name
# )

CPU times: total: 12.1 s
Wall time: 42.6 s


# 2. 제공되는 prompt를 활용하여 답변 생성

In [31]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [32]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [33]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

ai_message = qa_chain.invoke({'query':query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 약 624만원입니다. 이는 5천만원 초과 4억 8천만원 이하 세율인 24%를 적용한 계산입니다. 그러나 정확한 세액은 공제 및 개인별 상황에 따라 달라질 수 있습니다.'}